In [1]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import count
from collections import Counter

spark = SparkSession.builder.appName('LeetCode').getOrCreate()

24/12/27 12:13:59 WARN Utils: Your hostname, madiv resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlo1)
24/12/27 12:13:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/27 12:14:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark_Transactions = spark.read.option('header', True).option('delimiter', ',').csv('Transactions.csv')
spark_Visits = spark.read.option('header', True).option('delimiter', ',').csv('Visits.csv')

pandas_Transactions = pd.read_csv('Transactions.csv')
pandas_Visits = pd.read_csv('Visits.csv')

spark_Transactions.show()
spark_Visits.show()

print(pandas_Transactions.head())
print(pandas_Visits.head())

+--------------+--------+------+
|transaction_id|visit_id|amount|
+--------------+--------+------+
|             2|       5|   310|
|             3|       5|   300|
|             9|       5|   200|
|            12|       1|   910|
|            13|       2|   970|
+--------------+--------+------+

+--------+-----------+
|visit_id|customer_id|
+--------+-----------+
|       1|         23|
|       2|          9|
|       4|         30|
|       5|         54|
|       6|         96|
|       7|         54|
|       8|         54|
+--------+-----------+

   transaction_id  visit_id  amount
0               2         5     310
1               3         5     300
2               9         5     200
3              12         1     910
4              13         2     970
   visit_id  customer_id
0         1           23
1         2            9
2         4           30
3         5           54
4         6           96


### SQL

In [3]:
def find_customers(visits: pyspark.sql.dataframe.DataFrame, 
                   transactions: pyspark.sql.dataframe.DataFrame) -> pyspark.sql.dataframe.DataFrame:
    visits.createOrReplaceTempView('Visits')
    transactions.createOrReplaceTempView('Transactions')
    
    sqlQuery = \
    '''
    SELECT 
        v.customer_id, 
        COUNT(*) AS count_no_trans 
    FROM Visits v 
    LEFT JOIN Transactions t ON
    v.visit_id = t.visit_id
    WHERE t.transaction_id IS NULL
    GROUP BY v.customer_id
    '''
    
    output = spark.sql(sqlQuery = sqlQuery)
    return output

output = find_customers(spark_Visits, spark_Transactions)
output.show()

+-----------+--------------+
|customer_id|count_no_trans|
+-----------+--------------+
|         54|             2|
|         30|             1|
|         96|             1|
+-----------+--------------+



### PySpark

In [4]:
def find_customers(visits: pyspark.sql.dataframe.DataFrame, 
                   transactions: pyspark.sql.dataframe.DataFrame) -> pyspark.sql.dataframe.DataFrame:
    output = visits.join(transactions, on = 'visit_id', how = 'left')\
                    .filter(transactions.transaction_id.isNull())\
                    .select('customer_id')\
                    .groupBy('customer_id')\
                    .agg(count('customer_id').alias('count_no_trans'))
    return output

output = find_customers(spark_Visits, spark_Transactions)
output.show()

+-----------+--------------+
|customer_id|count_no_trans|
+-----------+--------------+
|         54|             2|
|         30|             1|
|         96|             1|
+-----------+--------------+



### Pandas

In [5]:
def find_customers(visits: pd.DataFrame, transactions: pd.DataFrame) -> pd.DataFrame:
    output = visits.merge(transactions, on = 'visit_id', how = 'left')
    output = output[output.transaction_id.isna()]
    output = Counter(output.customer_id)
    return pd.DataFrame(output.items(), columns = ['customer_id', 'count_no_trans'])

output = find_customers(pandas_Visits, pandas_Transactions)
print(output.head())

def find_customers(visits: pd.DataFrame, transactions: pd.DataFrame) -> pd.DataFrame:
    output = visits.merge(transactions, on = 'visit_id', how = 'left')
    output = output[output.transaction_id.isna()]
    output = output.groupby('customer_id').size().reset_index(name = 'count_no_trans')
    return output
    
output = find_customers(pandas_Visits, pandas_Transactions)
output.head()

   customer_id  count_no_trans
0           30               1
1           96               1
2           54               2


,customer_id,count_no_trans
0,30,1
1,54,2
2,96,1
